# Librerías

In [2]:
import pandas as pd
import numpy as np
import os
import shutil
import glob
from collections import defaultdict
import matplotlib as plt
import logging
from datetime import datetime
import random

# Definición de funciones

In [ ]:
# 1. Carga y Procesamiento de Anotaciones
def cargar_anotaciones(ruta_excel: str) -> pd.DataFrame:
    try:
        df = pd.read_excel(ruta_excel)
        columnas_numericas = ['xmin', 'ymin', 'xmax', 'ymax']
        df[columnas_numericas] = df[columnas_numericas].apply(pd.to_numeric, errors='coerce')
        df['class'] = df['class'].astype(str).str.strip().replace('nan', 'unknown')
        df = df.dropna(subset=columnas_numericas)
        return df
    except Exception as e:
        logging.error(f"Error cargando anotaciones: {str(e)}")
        return pd.DataFrame()

# 2. Cálculo de Áreas
def calcular_area(row: pd.Series) -> float:
    try:
        width = abs(row['xmax'] - row['xmin'])
        height = abs(row['ymax'] - row['ymin'])
        return width * height
    except Exception as e:
        logging.warning(f"Error calculando área para {row['filename']}: {str(e)}")
        return 0.0

# 3. Determinación de Clase Dominante
def determinar_clase_dominante(imagen_areas: dict) -> str:
    try:
        if not imagen_areas:
            return 'unknown'
        
        clases_ordenadas = sorted(
            imagen_areas.items(),
            key=lambda x: (-x[1], x[0])
        )
        
        max_area = clases_ordenadas[0][1]
        clases_maximas = [clase for clase, area in clases_ordenadas if area == max_area]
        
        if len(clases_maximas) > 1:
            return 'fish' if 'fish' in clases_maximas else sorted(clases_maximas)[0]
        return clases_ordenadas[0][0]
    except Exception as e:
        logging.error(f"Error determinando clase dominante: {str(e)}")
        return 'unknown'

# 4. Estructura de Directorios
def crear_estructura_directorios(clases: set, carpeta_destino: str):
    try:
        for clase in clases:
            clase_dir = os.path.join(carpeta_destino, str(clase))
            os.makedirs(clase_dir, exist_ok=True)
        logging.info("Estructura de directorios creada exitosamente")
    except Exception as e:
        logging.error(f"Error creando directorios: {str(e)}")

# 5. Procesamiento de Imágenes
def procesar_imagenes(df: pd.DataFrame, carpeta_origen: str, carpeta_destino: str, modo: str = 'copiar'):
    try:
        imagen_areas = defaultdict(lambda: defaultdict(float))
        
        # Calcular áreas acumuladas por imagen/clase
        for _, fila in df.iterrows():
            nombre_imagen = fila['filename']
            clase = fila['class'] or 'unknown'
            area = calcular_area(fila)
            imagen_areas[nombre_imagen][clase] += area
        
        # Determinar clase dominante
        imagen_clase = {img: determinar_clase_dominante(areas) for img, areas in imagen_areas.items()}
        
        # Crear directorios
        clases_unicas = set(imagen_clase.values()) | {'unknown'}
        crear_estructura_directorios(clases_unicas, carpeta_destino)
        
        # Mover/Copiar archivos
        for imagen, clase in imagen_clase.items():
            origen = os.path.join(carpeta_origen, imagen)
            destino = os.path.join(carpeta_destino, clase, imagen)
            
            if not os.path.exists(origen):
                logging.warning(f"Archivo no encontrado: {origen}")
                continue
            
            try:
                if modo == 'copiar':
                    shutil.copy(origen, destino)
                else:
                    shutil.move(origen, destino)
                logging.info(f"Archivo {'copiado' if modo == 'copiar' else 'movido'}: {destino}")
            except Exception as e:
                logging.error(f"Error procesando {imagen}: {str(e)}")
                
        return imagen_clase
    except Exception as e:
        logging.error(f"Error en procesamiento principal: {str(e)}")
        return {}
    
# 6. Distribuir las imágenes en las diferentes carpetas 
def distribuir_imagenes():
    # Configuración
    carpeta_principal = "imagenes_clasificadas"
    carpetas_destino = ["imagenes_train", "imagenes_test", "imagenes_validacion"]
    proporciones = [0.7, 0.15, 0.15]  # train, test, validation
    
    logging.info("Iniciando distribución de imágenes...")
    
    # Verificar carpeta principal
    if not os.path.exists(carpeta_principal):
        logging.error(f"La carpeta '{carpeta_principal}' no existe.")
        return False
    
    # Obtener clases (subcarpetas)
    clases = [d for d in os.listdir(carpeta_principal) if os.path.isdir(os.path.join(carpeta_principal, d))]
    if not clases:
        logging.error("No se encontraron clases en la carpeta principal.")
        return False
    
    logging.info(f"Se encontraron {len(clases)} clases: {clases}")
    
    # Crear carpetas de destino
    for carpeta in carpetas_destino:
        for clase in clases:
            os.makedirs(os.path.join(carpeta, clase), exist_ok=True)
    
    logging.info("Carpetas de destino creadas correctamente.")
    
    # Distribuir imágenes
    resultados = {destino: {clase: 0 for clase in clases} for destino in carpetas_destino}
    extensiones = ["*.jpg", "*.jpeg", "*.png", "*.gif", "*.bmp", "*.tiff"]
    
    for clase in clases:
        # Obtener todas las imágenes de esta clase
        imagenes = []
        for extension in extensiones:
            imagenes.extend(glob.glob(os.path.join(carpeta_principal, clase, extension)))
        
        if not imagenes:
            logging.warning(f"No se encontraron imágenes para la clase '{clase}'.")
            continue
        
        # Mezclar aleatoriamente
        random.shuffle(imagenes)
        
        # Calcular límites para cada conjunto
        total = len(imagenes)
        limite_train = int(total * proporciones[0])
        limite_test = limite_train + int(total * proporciones[1])
        
        # Distribuir imágenes
        for i, imagen in enumerate(imagenes):
            if i < limite_train:
                destino = carpetas_destino[0]  # train
            elif i < limite_test:
                destino = carpetas_destino[1]  # test
            else:
                destino = carpetas_destino[2]  # validation
            
            # Copiar imagen
            nombre_archivo = os.path.basename(imagen)
            destino_final = os.path.join(destino, clase, nombre_archivo)
            shutil.copy2(imagen, destino_final)
            resultados[destino][clase] += 1
        
        logging.info(f"Clase '{clase}': {total} imágenes distribuidas")
    
    # Mostrar resultados
    logging.info("Distribución completada:")
    for destino in carpetas_destino:
        total_destino = sum(resultados[destino].values())
        logging.info(f"  {destino}: {total_destino} imágenes")
        for clase, cantidad in resultados[destino].items():
            logging.info(f"    - {clase}: {cantidad} imágenes")
    
    
    logging.info("Proceso completado con éxito!")
    return True


# Cogemos las imagenes de la carpeta original y las clasificamos en clases

In [ ]:
def main():
    # Configuración
    RUTA_EXCEL = "annotations.xlsx"
    CARPETA_ORIGEN = "imagenes_originales"
    CARPETA_DESTINO = "imagenes_clasificadas"
    MODO = 'copiar'  # 'copiar' o 'mover'
    
    # Ejecución
    df = cargar_anotaciones(RUTA_EXCEL)
    if not df.empty:
        procesar_imagenes(df, CARPETA_ORIGEN, CARPETA_DESTINO, MODO)

if __name__ == "__main__":
    main()

# Distribuimos en las carpetas de Test, Train y Validacion las imágenes ya clasificadas

In [5]:
if __name__ == "__main__":
    distribuir_imagenes()